In [1]:
# --- Imports & config ---
import os, math, random, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix,
    mean_absolute_error, mean_squared_error
)
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import StandardScaler

# Reproducibility
SEED = 4120
np.random.seed(SEED)
random.seed(SEED)

# Paths
DATA_PATH = "../data/insurance.csv"
PLOTS_DIR = "../plots"
os.makedirs(PLOTS_DIR, exist_ok=True)

sns.set(context="notebook", style="whitegrid")
warnings.filterwarnings("ignore")

# --- 1. Load data and create targets (same logic as midpoint) ---

df = pd.read_csv(DATA_PATH)

# enforce dtypes
cat_cols = ["sex", "smoker", "region"]
for c in cat_cols:
    df[c] = df[c].astype("category")

# engineered features
df["bmi_obese"] = (df["bmi"] >= 30).astype(int)
df["age_band"]  = pd.cut(
    df["age"],
    bins=[17, 25, 35, 45, 55, 65],
    labels=["18-25","26-35","36-45","46-55","56-65"],
    right=True
)

# split once
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=SEED, shuffle=True)
val_df,   test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED, shuffle=True)

train_median = train_df["charges"].median()

def add_targets(d, median):
    d = d.copy()
    d["y_class"] = (d["charges"] >= median).astype(int)   # high vs low cost
    d["y_reg"]   = d["charges"].astype(float)
    return d

train_df = add_targets(train_df, train_median)
val_df   = add_targets(val_df,   train_median)
test_df  = add_targets(test_df,  train_median)

print("Train/Val/Test sizes:", len(train_df), len(val_df), len(test_df))
print("Train median charges:", round(train_median, 2))

# --- 2. Preprocessing for NN (fit on train, transform to dense arrays) ---

num_cols = ["age", "bmi", "children", "bmi_obese"]
cat_cols = ["sex", "smoker", "region", "age_band"]

numeric_tf     = StandardScaler()
categorical_tf = OneHotEncoder(handle_unknown="ignore")

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, num_cols),
        ("cat", categorical_tf, cat_cols),
    ]
)

X_train_raw = train_df[num_cols + cat_cols]
X_val_raw   = val_df[num_cols + cat_cols]
X_test_raw  = test_df[num_cols + cat_cols]

y_train_class = train_df["y_class"].values
y_val_class   = val_df["y_class"].values
y_test_class  = test_df["y_class"].values

y_train_reg = train_df["y_reg"].values
y_val_reg   = val_df["y_reg"].values
y_test_reg  = test_df["y_reg"].values

y_scaler = StandardScaler()
y_train_reg_scaled = y_scaler.fit_transform(y_train_reg.reshape(-1, 1)).ravel()
y_val_reg_scaled   = y_scaler.transform(y_val_reg.reshape(-1, 1)).ravel()
y_test_reg_scaled  = y_scaler.transform(y_test_reg.reshape(-1, 1)).ravel()

# fit on train, transform all
X_train = preprocess.fit_transform(X_train_raw)
X_val   = preprocess.transform(X_val_raw)
X_test  = preprocess.transform(X_test_raw)

# make sure dense
if hasattr(X_train, "toarray"):
    X_train = X_train.toarray()
    X_val   = X_val.toarray()
    X_test  = X_test.toarray()

print("Feature matrix shape:", X_train.shape)

# --- 3. Helper: training loop for MLPClassifier (classification NN) ---

def train_mlp_classifier(
    X_tr, y_tr, X_val, y_val,
    hidden_layer_sizes=(64, 32),
    alpha=1e-3,
    lr=1e-3,
    epochs=80,
):
    clf = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation="relu",
        solver="adam",
        alpha=alpha,
        learning_rate_init=lr,
        max_iter=1,           # we will loop manually
        warm_start=True,
        random_state=SEED,
    )
    classes = np.unique(y_tr)

    train_f1_list = []
    val_f1_list   = []
    train_acc_list = []
    val_acc_list   = []

    for epoch in range(epochs):
        clf.partial_fit(X_tr, y_tr, classes=classes)

        # metrics on train
        y_tr_pred = clf.predict(X_tr)
        train_f1  = f1_score(y_tr, y_tr_pred)
        train_acc = accuracy_score(y_tr, y_tr_pred)

        # metrics on val
        y_val_pred = clf.predict(X_val)
        val_f1  = f1_score(y_val, y_val_pred)
        val_acc = accuracy_score(y_val, y_val_pred)

        train_f1_list.append(train_f1)
        val_f1_list.append(val_f1)
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)

    history = {
        "train_f1": train_f1_list,
        "val_f1":   val_f1_list,
        "train_acc": train_acc_list,
        "val_acc":   val_acc_list,
    }
    return clf, history

# --- 4. Helper: training loop for MLPRegressor (regression NN) ---

def train_mlp_regressor(
    X_tr, y_tr_scaled, X_val, y_val_scaled,
    y_tr_orig, y_val_orig,
    hidden_layer_sizes=(64, 32),
    alpha=1e-3,
    lr=1e-3,
    epochs=80,
    y_scaler=None,
):
    reg = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation="relu",
        solver="adam",
        alpha=alpha,
        learning_rate_init=lr,
        max_iter=1,
        warm_start=True,
        random_state=SEED,
    )

    train_rmse_list = []
    val_rmse_list   = []

    for epoch in range(epochs):
        # Train on scaled targets
        reg.partial_fit(X_tr, y_tr_scaled)

        # Predictions in scaled space
        y_tr_pred_scaled  = reg.predict(X_tr)
        y_val_pred_scaled = reg.predict(X_val)

        # Convert back to original dollar scale for metrics
        if y_scaler is not None:
            y_tr_pred = y_scaler.inverse_transform(
                y_tr_pred_scaled.reshape(-1, 1)
            ).ravel()
            y_val_pred = y_scaler.inverse_transform(
                y_val_pred_scaled.reshape(-1, 1)
            ).ravel()
        else:
            y_tr_pred = y_tr_pred_scaled
            y_val_pred = y_val_pred_scaled

        # ✅ Use y_tr_orig and y_val_orig here (NOT y_tr / y_val)
        train_rmse = math.sqrt(mean_squared_error(y_tr_orig, y_tr_pred))
        val_rmse   = math.sqrt(mean_squared_error(y_val_orig, y_val_pred))

        train_rmse_list.append(train_rmse)
        val_rmse_list.append(val_rmse)

    history = {
        "train_rmse": train_rmse_list,
        "val_rmse":   val_rmse_list,
    }
    return reg, history


# --- 5. Small hyperparameter grids (very small, but enough for report) ---

clf_configs = [
    {"name": "clf_small",  "hidden": (32, 16), "alpha": 1e-3, "lr": 1e-3},
    {"name": "clf_medium", "hidden": (64, 32), "alpha": 1e-3, "lr": 5e-4},
]

reg_configs = [
    {"name": "reg_small",  "hidden": (32, 16), "alpha": 1e-3, "lr": 1e-3},
    {"name": "reg_medium", "hidden": (64, 32), "alpha": 1e-3, "lr": 5e-4},
]

best_clf = None
best_clf_hist = None
best_clf_name = None
best_clf_val_f1 = -1

print("\n=== Training classification NNs ===")
for cfg in clf_configs:
    print(f"Config {cfg['name']}: hidden={cfg['hidden']}, alpha={cfg['alpha']}, lr={cfg['lr']}")
    model, hist = train_mlp_classifier(
        X_train, y_train_class,
        X_val,   y_val_class,
        hidden_layer_sizes=cfg["hidden"],
        alpha=cfg["alpha"],
        lr=cfg["lr"],
        epochs=80,
    )
    final_val_f1 = hist["val_f1"][-1]
    print(f"  Final val F1 = {final_val_f1:.4f}")
    if final_val_f1 > best_clf_val_f1:
        best_clf_val_f1 = final_val_f1
        best_clf = model
        best_clf_hist = hist
        best_clf_name = cfg["name"]

print(f"\nBest classification NN: {best_clf_name} with final val F1 = {best_clf_val_f1:.4f}")

best_reg = None
best_reg_hist = None
best_reg_name = None
best_reg_val_rmse = float("inf")

print("\n=== Training regression NNs ===")
for cfg in reg_configs:
    print(f"Config {cfg['name']}: hidden={cfg['hidden']}, alpha={cfg['alpha']}, lr={cfg['lr']}")
    model, hist = train_mlp_regressor(
        X_train, y_train_reg_scaled,
        X_val,   y_val_reg_scaled,
        y_train_reg, y_val_reg,
        hidden_layer_sizes=cfg["hidden"],
        alpha=cfg["alpha"],
        lr=cfg["lr"],
        epochs=80,
        y_scaler=y_scaler,
    )
    final_val_rmse = hist["val_rmse"][-1]
    print(f"  Final val RMSE = {final_val_rmse:.2f}")
    if final_val_rmse < best_reg_val_rmse:
        best_reg_val_rmse = final_val_rmse
        best_reg = model
        best_reg_hist = hist
        best_reg_name = cfg["name"]

print(f"\nBest regression NN: {best_reg_name} with final val RMSE = {best_reg_val_rmse:.2f}")

# --- 6. Plot learning curves (required Plot 1 & Plot 2) ---

# Plot 1 – classification NN learning curve (train/val F1 vs epochs)
epochs = range(1, len(best_clf_hist["train_f1"]) + 1)
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(epochs, best_clf_hist["train_f1"], label="Train F1")
ax.plot(epochs, best_clf_hist["val_f1"],   label="Val F1")
ax.set_xlabel("Epoch")
ax.set_ylabel("F1 score")
ax.set_title(f"Classification NN learning curve ({best_clf_name})")
ax.legend()
plt.tight_layout()
plot1_path = os.path.join(PLOTS_DIR, "plot1_nn_class_learning_curve.png")
plt.savefig(plot1_path, dpi=160)
plt.close()
print(f"Saved Plot 1 (classification learning curve) to {plot1_path}")

# Plot 2 – regression NN learning curve (train/val RMSE vs epochs)
epochs = range(1, len(best_reg_hist["train_rmse"]) + 1)
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(epochs, best_reg_hist["train_rmse"], label="Train RMSE")
ax.plot(epochs, best_reg_hist["val_rmse"],   label="Val RMSE")
ax.set_xlabel("Epoch")
ax.set_ylabel("RMSE")
ax.set_title(f"Regression NN learning curve ({best_reg_name})")
ax.legend()
plt.tight_layout()
plot2_path = os.path.join(PLOTS_DIR, "plot2_nn_reg_learning_curve.png")
plt.savefig(plot2_path, dpi=160)
plt.close()
print(f"Saved Plot 2 (regression learning curve) to {plot2_path}")

# --- 7. Final NN metrics on validation and test (for report tables) ---

# Classification NN metrics
y_val_pred_clf  = best_clf.predict(X_val)
y_test_pred_clf = best_clf.predict(X_test)

try:
    y_val_prob_clf  = best_clf.predict_proba(X_val)[:,1]
    y_test_prob_clf = best_clf.predict_proba(X_test)[:,1]
except Exception:
    y_val_prob_clf  = y_val_pred_clf
    y_test_prob_clf = y_test_pred_clf

nn_clf_val_accuracy = accuracy_score(y_val_class, y_val_pred_clf)
nn_clf_val_f1       = f1_score(y_val_class, y_val_pred_clf)
nn_clf_val_roc_auc  = roc_auc_score(y_val_class, y_val_prob_clf)

nn_clf_test_accuracy = accuracy_score(y_test_class, y_test_pred_clf)
nn_clf_test_f1       = f1_score(y_test_class, y_test_pred_clf)
nn_clf_test_roc_auc  = roc_auc_score(y_test_class, y_test_prob_clf)

print("\n=== NN classification metrics ===")
print(f"Val  - Acc: {nn_clf_val_accuracy:.4f}, F1: {nn_clf_val_f1:.4f}, ROC-AUC: {nn_clf_val_roc_auc:.4f}")
print(f"Test - Acc: {nn_clf_test_accuracy:.4f}, F1: {nn_clf_test_f1:.4f}, ROC-AUC: {nn_clf_test_roc_auc:.4f}")

# Regression NN metrics (convert predictions back to original scale)
y_val_pred_reg_scaled  = best_reg.predict(X_val)
y_test_pred_reg_scaled = best_reg.predict(X_test)

y_val_pred_reg = y_scaler.inverse_transform(
    y_val_pred_reg_scaled.reshape(-1, 1)
).ravel()
y_test_pred_reg = y_scaler.inverse_transform(
    y_test_pred_reg_scaled.reshape(-1, 1)
).ravel()

nn_reg_val_mae  = mean_absolute_error(y_val_reg, y_val_pred_reg)
nn_reg_val_rmse = math.sqrt(mean_squared_error(y_val_reg, y_val_pred_reg))

nn_reg_test_mae  = mean_absolute_error(y_test_reg, y_test_pred_reg)
nn_reg_test_rmse = math.sqrt(mean_squared_error(y_test_reg, y_test_pred_reg))

print("\n=== NN regression metrics ===")
print(f"Val  - MAE: {nn_reg_val_mae:.2f}, RMSE: {nn_reg_val_rmse:.2f}")
print(f"Test - MAE: {nn_reg_test_mae:.2f}, RMSE: {nn_reg_test_rmse:.2f}")


# also store in a small dict for convenience (you don't have to use this, it's for us later)
nn_results = {
    "clf": {
        "val_accuracy": nn_clf_val_accuracy,
        "val_f1": nn_clf_val_f1,
        "val_roc_auc": nn_clf_val_roc_auc,
        "test_accuracy": nn_clf_test_accuracy,
        "test_f1": nn_clf_test_f1,
        "test_roc_auc": nn_clf_test_roc_auc,
    },
    "reg": {
        "val_mae": nn_reg_val_mae,
        "val_rmse": nn_reg_val_rmse,
        "test_mae": nn_reg_test_mae,
        "test_rmse": nn_reg_test_rmse,
    }
}

nn_results


Train/Val/Test sizes: 936 201 201
Train median charges: 9545.63
Feature matrix shape: (936, 17)

=== Training classification NNs ===
Config clf_small: hidden=(32, 16), alpha=0.001, lr=0.001
  Final val F1 = 0.9215
Config clf_medium: hidden=(64, 32), alpha=0.001, lr=0.0005
  Final val F1 = 0.9110

Best classification NN: clf_small with final val F1 = 0.9215

=== Training regression NNs ===
Config reg_small: hidden=(32, 16), alpha=0.001, lr=0.001
  Final val RMSE = 4521.07
Config reg_medium: hidden=(64, 32), alpha=0.001, lr=0.0005
  Final val RMSE = 4370.82

Best regression NN: reg_medium with final val RMSE = 4370.82
Saved Plot 1 (classification learning curve) to ../plots/plot1_nn_class_learning_curve.png
Saved Plot 2 (regression learning curve) to ../plots/plot2_nn_reg_learning_curve.png

=== NN classification metrics ===
Val  - Acc: 0.9254, F1: 0.9215, ROC-AUC: 0.9620
Test - Acc: 0.9154, F1: 0.9091, ROC-AUC: 0.9432

=== NN regression metrics ===
Val  - MAE: 2543.74, RMSE: 4370.82
Tes

{'clf': {'val_accuracy': 0.9253731343283582,
  'val_f1': 0.9214659685863874,
  'val_roc_auc': 0.961970570689998,
  'test_accuracy': 0.9154228855721394,
  'test_f1': 0.9090909090909091,
  'test_roc_auc': 0.943154761904762},
 'reg': {'val_mae': 2543.7399166354776,
  'val_rmse': 4370.820926757592,
  'test_mae': 2628.0644409321676,
  'test_rmse': 5046.45493640548}}

In [2]:
# --- Plot 3: Confusion matrix for best final classification model (NN) ---

from sklearn.metrics import ConfusionMatrixDisplay

cm_nn = confusion_matrix(y_test_class, y_test_pred_clf)

fig, ax = plt.subplots(figsize=(4.5, 4))
disp = ConfusionMatrixDisplay(confusion_matrix=cm_nn)
disp.plot(cmap="Blues", ax=ax, colorbar=False)

ax.set_title(f"Confusion Matrix (test) — NN classifier ({best_clf_name})")
ax.set_xlabel("Predicted label")
ax.set_ylabel("True label")

plt.tight_layout()
plot3_path = os.path.join(PLOTS_DIR, "plot3_confusion_matrix_best_nn.png")
plt.savefig(plot3_path, dpi=160)
plt.close()

print(f"Saved Plot 3 (confusion matrix, best NN classifier) to {plot3_path}")


Saved Plot 3 (confusion matrix, best NN classifier) to ../plots/plot3_confusion_matrix_best_nn.png


In [3]:
# --- Plot 4: Residuals vs predicted for best final regression model (NN regressor) ---

residuals_nn = y_test_reg - y_test_pred_reg  # true - predicted

fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(y_test_pred_reg, residuals_nn, s=12, alpha=0.7)
ax.axhline(0, linestyle="--", linewidth=1)

ax.set_xlabel("Predicted charges (NN)")
ax.set_ylabel("Residuals (true - predicted)")
ax.set_title(f"Residuals vs Predicted (test) — NN regressor ({best_reg_name})")

plt.tight_layout()
plot4_path = os.path.join(PLOTS_DIR, "plot4_residuals_best_nn_reg.png")
plt.savefig(plot4_path, dpi=160)
plt.close()

print(f"Saved Plot 4 (residuals vs predicted, best NN regressor) to {plot4_path}")


Saved Plot 4 (residuals vs predicted, best NN regressor) to ../plots/plot4_residuals_best_nn_reg.png


In [4]:
# --- Plot 5: Feature importance via permutation importance (Decision Tree regressor baseline) ---

from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import permutation_importance

# 1. Fit a Decision Tree regressor on the same preprocessed features
tree_reg_baseline = DecisionTreeRegressor(random_state=SEED)
tree_reg_baseline.fit(X_train, y_train_reg)

# 2. Build feature names from numeric + one-hot categorical
num_feature_names = num_cols  # ["age", "bmi", "children", "bmi_obese"]

cat_onehot_names = preprocess.named_transformers_["cat"].get_feature_names_out(cat_cols)
feature_names = list(num_feature_names) + list(cat_onehot_names)

# 3. Permutation importance on test set
perm = permutation_importance(
    tree_reg_baseline,
    X_test,
    y_test_reg,
    n_repeats=30,
    random_state=SEED,
    n_jobs=-1,
)

importances = perm.importances_mean
stds = perm.importances_std

# 4. Put into a DataFrame and sort
feat_imp_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances,
    "std": stds,
}).sort_values("importance", ascending=False)

print("Top 10 features by permutation importance (tree regressor):")
display(feat_imp_df.head(10))

# 5. Plot (bar plot of top 10)
top_k = 10
top_feats = feat_imp_df.head(top_k).iloc[::-1]  # reverse for nicer horizontal bar order

fig, ax = plt.subplots(figsize=(7, 5))
ax.barh(top_feats["feature"], top_feats["importance"])
ax.set_xlabel("Mean decrease in performance (permutation importance)")
ax.set_title("Permutation importance (Decision Tree regressor baseline)")

plt.tight_layout()
plot5_path = os.path.join(PLOTS_DIR, "plot5_feature_importance_perm_tree_reg.png")
plt.savefig(plot5_path, dpi=160)
plt.close()

print(f"Saved Plot 5 (permutation importance, tree regressor) to {plot5_path}")


Top 10 features by permutation importance (tree regressor):


,feature,importance,std
6,smoker_no,1.560286,0.154582
1,bmi,0.357972,0.083787
0,age,0.244631,0.056047
2,children,0.046630,0.031210
9,region_northwest,0.030525,0.021159
8,region_northeast,0.022490,0.029340
16,age_band_56-65,0.000659,0.000952
13,age_band_26-35,0.000527,0.001476
14,age_band_36-45,0.000103,0.000335
7,smoker_yes,0.000000,0.000000


Saved Plot 5 (permutation importance, tree regressor) to ../plots/plot5_feature_importance_perm_tree_reg.png


In [5]:
# === FINAL REPORT TABLES (Classification + Regression) ===

# --- Classification comparison table ---
clf_table = pd.DataFrame({
    "model": ["logreg (best classical)", "nn_clf_small (best NN)"],
    "val_accuracy": [0.92, nn_clf_val_accuracy],
    "val_f1": [0.92, nn_clf_val_f1],
    "val_roc_auc": [0.95, nn_clf_val_roc_auc],
    "test_accuracy": [0.92, nn_clf_test_accuracy],
    "test_f1": [0.92, nn_clf_test_f1],
    "test_roc_auc": [0.96, nn_clf_test_roc_auc],
})

clf_table = clf_table.round(2)

# --- Regression comparison table ---
reg_table = pd.DataFrame({
    "model": ["lin_reg (best classical)", "nn_reg (best NN)"],
    "val_mae": [4261.04, nn_reg_val_mae],
    "val_rmse": [5858.81, nn_reg_val_rmse],
    "test_mae": [4309.84, nn_reg_test_mae],
    "test_rmse": [6155.43, nn_reg_test_rmse],
})

reg_table = reg_table.round(2)

# Display both tables
print("=== FINAL CLASSIFICATION TABLE ===")
display(clf_table)

print("\n=== FINAL REGRESSION TABLE ===")
display(reg_table)

# Save to CSV (optional but recommended)
clf_table.to_csv("../plots/final_table_classification.csv", index=False)
reg_table.to_csv("../plots/final_table_regression.csv", index=False)

print("\nSaved final tables to ../plots/")


=== FINAL CLASSIFICATION TABLE ===


,model,val_accuracy,val_f1,val_roc_auc,test_accuracy,test_f1,test_roc_auc
0,logreg (best classical),0.92,0.92,0.95,0.92,0.92,0.96
1,nn_clf_small (best NN),0.93,0.92,0.96,0.92,0.91,0.94



=== FINAL REGRESSION TABLE ===


,model,val_mae,val_rmse,test_mae,test_rmse
0,lin_reg (best classical),4261.04,5858.81,4309.84,6155.43
1,nn_reg (best NN),2543.74,4370.82,2628.06,5046.45



Saved final tables to ../plots/
